In [0]:
import os
import urllib.request
from pyspark.sql import SparkSession

In [0]:
# --- PART 1: The Setup 🛠️ ---
# We configure Spark to be ready for some heavy lifting
spark = SparkSession.builder.appName("CreditRiskIngestion").getOrCreate()
# --- PART 2: The Download (Python Layer) 🐍 ---
# URL to a public subset of LendingClub data (hosted on GitHub for easy access)
# Note: Real LendingClub data is huge, this is a workable subset for our demo.
dataset_url = "https://raw.githubusercontent.com/anushkaparadkar/lending-club-case-study/master/loan.csv"
local_file_name = "/Volumes/workspace/azure_data/az/lending_club_loans.csv"

print(f"Downloading dataset from {dataset_url}")
# We use urllib to fetch the file to our local environment
try:
    if not os.path.exists(local_file_name):
        urllib.request.urlretrieve(dataset_url,local_file_name)
        print("Download complete")
    else:
        print("File already exists, skipping download")
except Exception as e:
    print(f"Error downloading {e}")


In [0]:
# 4. Load Data into PySpark DataFrame
# inferSchema=True is useful but can be slow on huge data; fine for this size (~40MB)
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(local_file_name)

In [0]:
# Basic Inspection
print(f"Rows: {df.count()}")
print(f"Columns: {len(df.columns)}")
df.printSchema()
display(df)

In [0]:
# Calculate the count of nulls for each column
# We iterate through columns to build an aggregation expression
from pyspark.sql import functions as F
null_counts = {
    c: df.filter(F.col(c).isNull()).count()
    for c in df.columns
}

In [0]:
# Calculate percentage of nulls
total_rows =df.count()
null_ratio = {k: v/total_rows for k , v in null_counts.items()}

In [0]:
# Filter columns that are mostly empty (e.g., > 50% null)
cols_to_drop = [k for k, v in null_ratio.items() if v > 0.5]
print(f"Dropping columns: {cols_to_drop}")
print(cols_to_drop)

In [0]:
#Drop them
df_cleaned = df.drop(*cols_to_drop)
print(f"new columns count: {len(df_cleaned.columns)}")

In [0]:
from pyspark.sql.functions import regexp_replace, col,trim
# 1. Fix 'int_rate': Remove '%' and cast to Double
df_cleaned = df_cleaned.withColumn("int_rate_cleaned",regexp_replace(col("int_rate"),"%","").cast("double"))

In [0]:
# 2. Fix 'term': Remove ' months' and cast to Integer
# We also use trim() because 'term' often has leading spaces like " 36 months"
df_cleaned = df_cleaned.withColumn("term_cleaned", regexp_replace(trim(col("term")),"months","").cast("int"))

In [0]:
df_cleaned.select("int_rate","int_rate_cleaned","term","term_cleaned").show(5)

In [0]:
# Check distinct values in loan_status before filtering
print("Before filtering")
df_cleaned.select("loan_status").distinct().show()

In [0]:
# Filter to keep only 'Fully Paid' and 'Charged Off'
# Note: Spark is case-sensitive, so ensure strings match exactly
df_final = df_cleaned.filter(col("loan_status").isin("Charged Off", "Fully Paid"))

#verify count after filtering
print(f"Final row count after filtering: {df_final.count()}")

In [0]:
from pyspark.sql.functions import when

# Create a new column 'loan_status_num'
df_final = df_final.withColumn("loan_satus_num", when(col("loan_status") == "Charged Off", 1).otherwise(0))
df_final.select("loan_status", "loan_satus_num").show(5)

In [0]:
from pyspark.sql.functions import avg, count, round
grade_analysis = df_final.groupBy("grade") \
    .agg(
        count("id").alias("Total_loans"), 
        round(avg("loan_satus_num")*100,2).alias("Default_rate_percent")
    ) \
    .orderBy("grade")
grade_analysis.show()

In [0]:
# Group by 'term_cleaned' with correct column name
term_analysis = df_final.groupBy("term_cleaned").agg(
    count("id").alias("total_loans"),
    round(avg("loan_satus_num") * 100, 2).alias("default_rate_percent")
).orderBy("term_cleaned")

display(term_analysis)

In [0]:
# Group by 'purpose' and sort by highest default risk
purpose_analysis = df_final.groupBy("purpose").agg(
    count("id").alias("total_loans"),
    round(avg("loan_satus_num") * 100, 2).alias("default_rate_percent")
).orderBy("default_rate_percent", ascending=False)

display(purpose_analysis)   

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
# Convert the small aggregated Spark DataFrame to Pandas
pdf = grade_analysis.toPandas()

#plot
plt.figure(figsize=(10,6))
sns.barplot(x="grade",y="Default_rate_percent", data=pdf)
plt.title("Default rate by Loan Grade", fontsize = 16)
plt.ylabel("Default Rate %")
plt.xlabel("Grade")
plt.show()

